## LSTM with Connectionist Temporal Classifier - Lasagne
**Description**: A script to test lasagne-rnn-ctc using toy ascii alphabet data from [rakeshvar's repo](https://github.com/rakeshvar/rnn_ctc). 
**Dependencies**: print_utils.py, data.pkl *(ascii scribe) from rakeshvar*


In [1]:
import pickle as pkl
import numpy as np
import time
import theano
import theano.tensor as T
import lasagne
import lasagne.layers as L
import ctc_cost

from lasagne.layers import InputLayer, LSTMLayer, ReshapeLayer, DenseLayer, NonlinearityLayer, GRULayer, GaussianNoiseLayer
from print_utils import slab_print, prediction_printer

floatX = theano.config.floatX

In [2]:
# Training Params 
LEARNING_RATE = 0.001
N_BATCH = 20
NUM_EPOCHS = 1

# Number of units in the hidden (recurrent) layer
L1_UNITS = 50
L2_UNITS = 100

#### Load Ascii Toy Data File from rakeshvar ####
filename = 'data.pkl'
with open(filename,"rb") as pkl_file:
	data = pkl.load(pkl_file)
    
chars = data['chars']
nClasses = len(chars)
nDims = len(data['x'][0])
nSamples = len(data['x'])
nTrainSamples = int(nSamples * .75)
labels_print, labels_len = prediction_printer(chars)  

data_x = []
data_y = []

for x, y in zip(data['x'], data['y']):
# Insert blanks at alternate locations in the labelling (blank is nClasses)
   y1 = [nClasses]
   for char in y:
       y1 += [char, nClasses]

   data_y.append(np.asarray(y1, dtype=np.int32))
   data_x.append(np.asarray(x,  dtype=theano.config.floatX))

The lists *data_x* and *data_y* are the training sequences and their corresponding labels. A sample sequence and label is printed below:

In [3]:
print("Printing sample input ...")
idx = 0 
slab_print(data_x[idx])
chars.append(' ') 
print(data_y[idx], "".join(chars[i] for i in data_y[idx]))

Printing sample input ...
 0¦                         ¦
 1¦               █         ¦
 2¦               █         ¦
 3¦    █    █     █         ¦
 4¦     █  █      █         ¦
 5¦      ██       █         ¦
 6¦    ██  ██     █         ¦
 7¦               █         ¦
 8¦                         ¦
 9¦                         ¦
(array([95, 88, 95, 92, 95], dtype=int32), ' x | ')


## Zero Padding  Training Data to fixed Length
Here, both input sequences and target labels are zero-padded to fixed maximum sequence lengths. 

In [4]:
# Convert list of input sequences to zero-padded 3D array
num_feat   = data_x[0].shape[0]
max_x_len  = np.max([bb.shape[1] for bb in data_x])  #list comprehension to get all lengths
x          = np.zeros([len(data_x), max_x_len, num_feat])
for i, examples in enumerate(data_x):
    for j, feat in enumerate(examples):
        for k, seq in enumerate(feat):
            x[i][k][j]=seq

            
# Convert list of target sequences to zero-padded 2D array
max_y_len = max(map(len,data_y))
y=np.zeros([len(data_y), max_y_len],dtype=np.int32)
for i, examples in enumerate(data_y):
    for j, seq in enumerate(examples):
        y[i][j]=seq            

## LSTM Network Architecture
And then to define the LSTM-RNN model:

In [5]:
num_batch     = None  # use none to enable variable batch size
input_seq_len = None  # use none to enable variable sequence length
num_feat      = num_feat
num_classes   = nClasses + 1

soft = lasagne.nonlinearities.softmax
tanh = lasagne.nonlinearities.tanh
identity = lasagne.nonlinearities.identity

l_in = InputLayer(shape=(num_batch, input_seq_len, num_feat))
batchsize, seqlen, _ = l_in.input_var.shape

l_noise = GaussianNoiseLayer(l_in, sigma=0.6) 
# l_mask  = InputLayer(shape=(batchsize, seqlen))
# l_rnn_1 = LSTMLayer(l_noise, num_units=L1_UNITS, mask_input=l_mask)
l_rnn_1 = LSTMLayer(l_noise, num_units=L1_UNITS)
l_rnn_2 = LSTMLayer(l_rnn_1, num_units=L2_UNITS)
l_shp   = ReshapeLayer(l_rnn_2,(-1, L2_UNITS))
l_out   = DenseLayer(l_shp, num_units=num_classes, nonlinearity=identity)
l_out_shp  = ReshapeLayer(l_out, (batchsize, seqlen, num_classes)) 

l_out_softmax = NonlinearityLayer(l_out, nonlinearity=soft)
l_out_softmax_shp = ReshapeLayer(l_out_softmax,(batchsize, seqlen, num_classes))

output_lin_ctc = L.get_output(l_out_shp)
network_output = L.get_output(l_out_softmax_shp)
all_params = L.get_all_params(l_rnn_2, trainable=True)

/Users/Junior/anaconda/lib/python2.7/site-packages/theano/scan_module/scan.py:1019: Warning: In the strict mode, all neccessary shared variables must be passed as a part of non_sequences
  'must be passed as a part of non_sequences', Warning)


## Costs, Gradients & Training Functions

In [6]:
# Cost functions
target_values = T.imatrix('target_output')
input_values  = T.imatrix()

### Gradients ###
# pseudo costs - ctc cross entropy b/n targets and linear output - used in training
pseudo_cost = ctc_cost.pseudo_cost(target_values, output_lin_ctc)
pseudo_cost_grad = T.grad(pseudo_cost.sum() / batchsize, all_params)
pseudo_cost = pseudo_cost.mean()

# true costs
cost = ctc_cost.cost(target_values, network_output)
cost = cost.mean()

# Compute SGD updates for training
print("Computing updates ...")
updates = lasagne.updates.rmsprop(pseudo_cost_grad, all_params, LEARNING_RATE)

# Theano functions for training and computing cost
print("Compiling functions ...")
train = theano.function(
    [l_in.input_var, target_values], [cost, pseudo_cost, network_output], updates=updates)
validate = theano.function([l_in.input_var, target_values], [cost, network_output]) 
predict  = theano.function([l_in.input_var], network_output)

Computing updates ...
Compiling functions ...


/Users/Junior/anaconda/lib/python2.7/site-packages/theano/scan_module/scan_perform_ext.py:135: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


## Network Training 

In [7]:
#### Training Network ####
print("Training network ...")
num_batches_train = int(np.ceil(len(x) / N_BATCH))
split_ratio = 0.7*num_batches_train

for epoch in range(NUM_EPOCHS):
	now = time.time
	tlosses = []
	vlosses = []
	plosses = []
	probabilities = []

	traindata = zip(x,y)
	np.random.shuffle(traindata)
	sequences, labels = zip(*traindata)


	for batch in range(num_batches_train):
		batch_slice = slice(N_BATCH * batch, N_BATCH * (batch + 1))
		xi = sequences[batch_slice]
		yi = labels[batch_slice]
		if batch < split_ratio:
			loss, ploss, probs = train(xi,yi)
			tlosses.append(loss)
			plosses.append(ploss)
		else:
			loss, probs = validate(xi,yi)
			y_pred = np.argmax(probs, axis=-1)                           
			vlosses.append(loss)
			probabilities.append(probs)

		print("Batch {0}/{1}, loss:{2:.6}, ploss:{3:.6}".format(batch,num_batches_train,loss,ploss))


Training network ...
Batch 0/50, loss:143.98, ploss:-0.022
Batch 1/50, loss:143.21, ploss:-0.796
Batch 2/50, loss:142.00, ploss:-2.082
Batch 3/50, loss:140.95, ploss:-3.232
Batch 4/50, loss:138.54, ploss:-5.657
Batch 5/50, loss:133.14, ploss:-11.38
Batch 6/50, loss:115.49, ploss:-29.26
Batch 7/50, loss:86.369, ploss:-54.46
Batch 8/50, loss:64.904, ploss:-60.02
Batch 9/50, loss:49.446, ploss:-48.14
Batch 10/50, loss:43.707, ploss:-34.34
Batch 11/50, loss:36.948, ploss:-24.17
Batch 12/50, loss:34.621, ploss:-19.48
Batch 13/50, loss:33.799, ploss:-17.15
Batch 14/50, loss:32.544, ploss:-15.40
Batch 15/50, loss:33.901, ploss:-13.39
Batch 16/50, loss:29.552, ploss:-15.50
Batch 17/50, loss:26.301, ploss:-16.19
Batch 18/50, loss:24.131, ploss:-21.04
Batch 19/50, loss:22.438, ploss:-18.18
Batch 20/50, loss:22.036, ploss:-20.31
Batch 21/50, loss:23.491, ploss:-12.35
Batch 22/50, loss:21.709, ploss:-17.10
Batch 23/50, loss:22.664, ploss:-12.40
Batch 24/50, loss:19.545, ploss:-15.31
Batch 25/50, l

In [8]:
# Print a test example
idx=0
slab_print(xi[idx].T)
print(yi[idx], "".join(chars[i] for i in yi[idx]))

 0¦                                     ¦
 1¦                                     ¦
 2¦         █                           ¦
 3¦        █                            ¦
 4¦       █                             ¦
 5¦       █                             ¦
 6¦        █                            ¦
 7¦         █                           ¦
 8¦                                     ¦
 9¦                                     ¦
(array([95,  8, 95,  0,  0,  0,  0], dtype=int32), ' (     ')


In [9]:
# Predict a test example 
probs=predict(xi)
y_pred=np.argmax(probs,axis=-1)
print(y_pred[idx], "".join(chars[i] for i in y_pred[idx]))

(array([95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
        0,  0,  0]), '                                     ')
